In [ ]:
# Preparamos todo para correr
import numpy as np
from math import sqrt
from matplotlib import pylab as plt
from scipy.stats import norm


# Probabilidad Condicionada y Teorema de Bayes

Apliquemos los conceptos vistos con Rodrigo en el siguiente problema:

__Ejercicio__: 

Una enfermedad genética rara afecta al 0.1% de la población de un país. Para diagnosticarla, existe un análisis clínico que detecta la enfermedad en el 99% de los casos en los que el paciente la padece, y tiene un 1% de falsos positivos.

Un paciente recibe un resultado positivo del análisis clínico.

¿Qué probabilidad hay de que la persona padezca la enfermedad?
¿Cuál sería el próximo paso natural?

Antes de avanzar, entendamos bien el problema.

__Cual es el espacio de muestreo?__

$S = (E,\bar{E})\cdot(D,\bar{D})$

$S = (ED,E\bar{D},\bar{E}D.\bar{E}\bar{D})$

__Que es un falso positivo?__

Falso Positivo: Paciente sano, test positivo $\rightarrow$ $\bar{E}D$ 

In [ ]:
pE=0.001
pDE=0.99
pDnE=0.01

__Escribamos las probabilidades que podemos inferir del enunciado__

Queremos $P(E | D)$

Sabemos que:

1.   $P(E) = 0.001$
2.   $P(D|E) = 0.99$
3.   $P(D|\bar{E}) = 0.01 $

$P(E|I) + P(\bar{E}|I) = 1 $

$P(D|I) + P(\bar{D}|I) = 1 $



Ahora si, resolvamos el problema. 

__Que queremos obtener? Como podemos obtenerlo a partir de los datos?__ 

__Teorema de Bayes__:

$P(E|D)=\frac{P(D|E)P(E)}{P(D)}$

__Prior:__ $P(E)$
__Verosimilitud:__ $P(D|E)$
__Evidencia:__ $P(D)$
__Posterior:__ $P(E|D)$

__Regla de la Suma:__

$P(D) = P(D|E)p(E) + P(D|\bar{E})p(\bar{E})$

In [ ]:
pD=pDE*pE+pDnE*(1-pE)
print("Probabilidad total de un test positivo: ", pD)

In [ ]:
pED=pDE*pE/pD
print("Probabilidad de estar enfermo dado que dio un test positivo: ", pED)

__Falso positivo:__

$P(\bar{E}|D)=1-P(E|D)$

__Testear una segunda calcular:__

$P(E|D_{1}D_{2})=\frac{P(D_{1}D_{2}|E)P(E)}{p(D_{1}D_{2})}$

Utilizo indepndencia de los test:

$p(D_{1}D_{2}|E)=p(D_{2}|D_{1}E)p(D_{1}|E)$

Como $p(D_{2}|D_{1}E)=p(D_{2}|E)$

$p(D_{1}D_{2}|E)=p(D|E)^2$

$P(D_{1}D_{2})=P(D_{1}D_{2}|E)p(E)+P(D_{1}D_{2}|\bar{E})p(\bar{E})$

$P(D_{1}D_{2})=p(D|E)^{2}p(E)+p(D|\bar{E})^{2}p(\bar{E})$

__Ojo:__

$P(D_{1}D_{2})\neq P(D_{1})P(D_{2})$

__Actualizacion Bayesiana:__

$P(E|D_{2}D_{1})=\frac{P(D_{2}|ED_{1})P(E|D_{1})}{P(D_{2}|D_{1})}$

# Inferencia Bayesiana y Prior conjugado

In [ ]:
norm?

Resolvamos el siguiente ejercicio

__Ejercicio__

Para mantener el distanciamiento social, decidi comprar un medidor de distancias a base de laser. Segun el fabricante, la incerteza en la medicion es de 1 cm (estoy exagerando, no culpen a los laseres). En la fila del supermercado, y como estoy aburrido, observe que no estoy del todo seguro de que la persona delante mio este a 1 metro y medio de distancia: puede estar 10 centrimetros mas adelante o mas atras (se me empanian los anteojos). Dado que llevo el laser conmigo, mido 5 veces la distancia para estar seguro.

Como puedo usar lo que vimos con Rodrigo para deducir la distancia?

Yo quiero la distancia verdadera $\mu_{true}$.

{$x_{1=i}$} $i = 1,...,5$

__Datos__

Voy a generar los datos yo. Utilizo el metodo norm.rvs

In [ ]:
N=5
mu_true=1.30
sigma=0.01
distancias=norm.rvs(loc=mu_true,scale=sigma,size=N)
plt.hist(distancias)

__Teorema de Bayes__

Planteemos el problema en funcion de parametros y mediciones. Veamos que queremos deducir.

$p(\mu_{true}|\vec{x}) = \frac{p(\vec{x}|\mu_{true})p(\mu_{true})}{p(\vec{x})}$



## Oculten esto hasta resolver el Teorema de Bayes

__Verosimilitud__

Como argumentamos, vamos a asumir que la distribucion de los datos, es decir las mediciones de la distancia es una Gaussiana dictada por la distancia real $\mu$ con incerteza $\sigma$ dada por el laser. En ese caso, para una unica medicion la verosimilitud tendra la forma

$p(x|\mu,\sigma)=\mathcal{N}(x|\mu,\sigma)=\frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

Si tengo N mediciones independientes:

$p(x_1,x_2,...,x_N|\mu,\sigma)=\prod_{i=1}^{N}p(x_{i}|\mu,\sigma)=\frac{1}{(2\pi\sigma^2)^{N/2}}e^{-\frac{1}{2\sigma^2}\sum_{i=1}^{N}(x_{i}-\mu)^2}$

In [ ]:
xaux=np.linspace(40.0,60.0,10)
print("Mis x:", xaux)
print("Mis pdf(x):", norm.pdf(xaux,loc=50.0,scale=5.0))
print(xaux[norm.pdf(xaux,loc=50.0,scale=5.0)==max(norm.pdf(xaux,loc=50.0,scale=5.0))])
plt.plot(xaux,norm.pdf(xaux,loc=50.0,scale=5.0))

__Prior__

Considero como prior para la distancia real $\mu$ una distribucion normal centrada en mi estimacion a ojo $\mu_{0}$ y la incerteza de esa estimacion $\sigma_{0}$.

$p(\mu|\mu_{0},\sigma_{0})=\mathcal{N}(\mu|\mu_{0},\sigma_{0})=\frac{1}{\sqrt{2\pi\sigma_{0}^{2}}}e^{-\frac{(\mu-\mu_{0})^2}{2\sigma_{0}^2}}$

In [ ]:
mu=np.linspace(1.0,2.0,100)
plt.plot(mu,norm.pdf(mu,loc=1.5,scale=0.1))
plt.xlabel('$\mu$')
plt.ylabel('$p(\mu|\mu_{0}=1.5,\sigma_{0}=0.1$')

__Prior conjugado:__

Si mi prior es $f(a_{0},b_{0},...,z_{0})$, mi posterior es la misma funcion f pero con parametros actualizados $f(a_{N},b_{N},...,z_{N})$

__Inferencia__

Este prior tiene la ventaja de que es conjugado. Esto quiere decir que, viendo el __Teorema de Bayes__

$p(\mu|x_{1},x_{2},...,x_{N}, \sigma, \mu_0,\sigma_0)=\frac{\mathcal{N}(x_1,x_2,...,x_N|\mu,\sigma)\mathcal{N}(\mu|\mu_{0},\sigma_{0})}{p(x_1,x_2,...,x_N|\sigma,\mu_0,\sigma_0)}=\mathcal{N}(\mu|\mu_N,\sigma_N)$

Donde 

$\mu_N = \frac{\sigma^2}{\sigma^2+N\sigma_{0}^{2}}\mu_{0}+\frac{N\sigma_{0}^2}{\sigma^2+N\sigma_{0}^{2}}\mu_{ML}$

$\frac{1}{\sigma_{N}^{2}}=\frac{N}{\sigma^{2}}+\frac{1}{\sigma_{0}^{2}}$

Con $\mu_{ML} = \frac{1}{N}\sum_{i=1}^{N}x_{i}$, sale de $\frac{d}{d\mu}p(\vec{x}|\mu)=0$

Es decir, uno puede resolver analiticamente este problema y obtener la distribucion de probabilidad de la distancia real que incorpora las $N$ mediciones.

__Mini ejercicio__

Charlemos limites de $\mu_N$ y $\sigma_N$

```
# Esto tiene formato de código
```



__Implementacion__

Hagamoslo de dos maneras: grafiquemos la funcion analitica y apliquemos el teorema de Bayes numericamente.

In [ ]:
mu =np.linspace(1.0,2.0,100)
mu_0=1.5
sigma_0=0.1
plt.plot(mu,norm.pdf(mu,loc=mu_0,scale=sigma_0),label='Prior')
verosimilitud_aux=np.zeros((len(distancias),len(mu)))
for i in range(len(distancias)):
  verosimilitud_aux[i]=norm.pdf(distancias[i],loc=mu,scale=sigma)
#verosimilitud_aux=np.asarray(list(map(lambda x: norm.pdf(x,loc=mu,scale=sigma),distancias)))
print(verosimilitud_aux.shape)
verosimilitud=np.prod(verosimilitud_aux,axis=0)
# print(verosimilitud.shape)
# ax = plt.gca()
# ax2 = ax.twinx()
# ax2.plot(mu,verosimilitud,label='Verosimilitud',color='Red')
mu_ML=np.mean(distancias)
mu_N = sigma**2/(sigma**2+N*sigma_0**2)*mu_0+N*sigma_0**2/(sigma**2+N*sigma_0**2)*mu_ML
sigma_N=sigma_0*sigma/np.sqrt(N*(sigma_0**2)+sigma**2)
posterior=norm.pdf(mu,loc=mu_N,scale=sigma_N)
plt.plot(mu,posterior,label='Posterior')
plt.legend(loc='upper right')
plt.xlabel('$\mu$')

In [ ]:
print((mu_0,mu_ML,mu_N))

### Implementacion hecha por si no alcanza el tiempo

__Analitica__

In [ ]:
mu_0=1.5
sigma_0=0.1
mu_ML=np.mean(distancias)
nsteps=100
mu=np.linspace(1.0,2.0,nsteps)
bin=(mu[-1]-mu[0])/nsteps
prior=norm(loc=mu_0,scale=sigma_0)
plt.plot(mu,prior.pdf(mu),color='blue',label='Prior')

verosimilitud_aux=list(map(lambda x: norm.pdf(x,mu,sigma*np.ones(len(mu))),distancias))
verosimilitud=np.prod(verosimilitud_aux,axis=0)
plt.plot(mu,verosimilitud/(bin*np.sum(verosimilitud)),color='red',label='Forma de la Verosimilitud')


mu_N=mu_0*(sigma**2)/(N*(sigma_0)**2+sigma**2)+mu_ML*(N*sigma_0**2)/(N*(sigma_0)**2+sigma**2)
sigma_N=np.sqrt(1/(1/sigma_0**2+N/sigma**2))
posterior_an=norm(loc=mu_N,scale=sigma_N)
plt.plot(mu,posterior_an.pdf(mu),color='magenta',label='Posterior Analtica')

plt.xlabel('$\mu$')
plt.legend(loc='upper left')


print(np.mean(posterior_an.rvs(size=nsteps)),np.sqrt(np.var(posterior_an.rvs(size=nsteps))))


__Numerica__

In [ ]:
mu_0=1.5
sigma_0=0.1
mu_ML=np.mean(distancias)
nsteps=100
mu=np.linspace(1.0,2.0,nsteps)
bin=(mu[-1]-mu[0])/nsteps
prior=norm(loc=mu_0,scale=sigma_0).pdf(mu)
# prior=np.zeros_like(mu)
# prior[::5] = 1
plt.plot(mu,prior,color='blue',label='Prior')

verosimilitud_aux=list(map(lambda x: norm.pdf(x,mu,sigma*np.ones(len(mu))),distancias))
verosimilitud=np.prod(verosimilitud_aux,axis=0)

print((verosimilitud*prior).shape)
# plt.plot(mu,verosimilitud/(bin*np.sum(verosimilitud)),color='red',label='Forma de la Verosimilitud')

# print((mu.shape,prior.pdf(mu).shape,verosimilitud.shape))
evidencia=np.sum(verosimilitud*prior)*bin
posterior_num=verosimilitud*prior/evidencia
plt.plot(mu,posterior_num,color='green',label='Posterior Numerica')

plt.xlabel('$\mu$')
plt.legend(loc='upper left')

print(np.sum(mu*posterior_num)*bin)
print(mu[np.argmax(posterior_num)])
#print(np.mean(posterior_an.rvs(size=nsteps)),np.sqrt(np.var(posterior_an.rvs(size=nsteps))))

### Demostracion 

Demostremos la propiedad del prior conjugado utilizando una unica medicion $x$. Para ese caso

$p(\mu|x,\sigma, \mu_0,\sigma_0)=\frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}\frac{1}{\sqrt{2\pi\sigma_{0}^{2}}}e^{-\frac{(\mu-\mu_{0})^2}{2\sigma_{0}^2}}\frac{1}{p(x|\sigma,\mu_{0},\sigma_{0})}$

Con $p(x|\sigma,\mu_{0},\sigma_{0})=\int\frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}\frac{1}{\sqrt{2\pi\sigma_{0}^{2}}}e^{-\frac{(\mu-\mu_{0})^2}{2\sigma_{0}^2}}d\mu $

Las constantes multiplicativas se simplifican. Ademas, puedo sacar todos los terminos que no dependan de $\mu$ de la integral y simplificarlos. Estos son

$e^{-\frac{x^2}{2\sigma^2}}e^{-\frac{\mu_{0}^2}{2\sigma_{0}^{2}}}$

Simplificando y juntando las exponenciales

$p(\mu|x,\sigma, \mu_0,\sigma_0)=\frac{e^{-[\mu^2(\frac{1}{2\sigma^2}+\frac{1}{2\sigma_{0}^2})-2\mu(\frac{x}{2\sigma^2}+\frac{\mu_0}{2\sigma_{0}^{2}})]}}{\int e^{-[\mu^2(\frac{1}{2\sigma^2}+\frac{1}{2\sigma_{0}^2})-2\mu(\frac{x}{2\sigma^2}+\frac{\mu_0}{2\sigma_{0}^{2}})]}d\mu }$

Recordemos que se cumple trivialmente que $\int p(\mu|x,\sigma,\mu_0,\sigma_0)d\mu =1$

Vemos que esto tiene forma de exponencial cuadratica en $\mu$. Recordando la forma generica de una gaussiana:

$\mathcal{N}(\mu|\mu_{1},\sigma_{1})=\frac{e^{-\frac{(\mu-\mu_{1})^2}{2\sigma_{1}^{2}}}}{\int e^{-\frac{(\mu-\mu_{1})^2}{2\sigma_{1}^{2}}} d\mu}$

Abriendo cuadrados obtenemos $\mu_{1}$ y $\sigma_{1}$ comparando termino por termino.  Noten que la expresion de $p(\mu|x,\sigma,\mu_{0},\sigma_{0})$ solo contiene terminos cuadraticos y lineales en $\mu$. Sin embargo, el termino $e^{-\frac{\mu_{1}^{2}}{2\sigma_{1}^2}}$ puede agregarse tanto al denominador como al numerador, recuperando efectivamente $\mathcal{N}(\mu|\mu_1,\sigma_1)$.

Para el caso de $N$ mediciones independientes, las cuentas son analogas y pueden obtener $\mu_{N}$ y $\sigma_{N}$.

